In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
pd.options.display.max_columns = 50

In [3]:
# Methods
def unpack_column(dataFrame,col_name):
    newDf = dataFrame.from_records(dataFrame[col_name].dropna().tolist())
    return newDf

def unpack_and_destroy(dataFrame,column):
    temp = unpack_column(dataFrame,column)
    for col in temp.columns:
        dataFrame[col] = temp[col]
    #dataFrame = dataFrame.drop(columns=[column])
    return dataFrame

def sort_db_value(db,value,asc=False):
    return db.sort_values(by = value, ascending = asc)

def freq_array(freq_list):
    freq = []
    for i in freq_list:
        freq.append(i)
    return freq

def set_labels(freq, ax,horz_shift=.2,vert_shift=1000,w="bold",fsize=10,c="b"):
    for i, v in enumerate(freq):
        ax.text(i-horz_shift,v+vert_shift, str(round(v,2)), weight = w,fontsize=fsize,color = c)
        
def wrap_axis_labels(labels_list,wrap_length=13):
    labels_list = [ '\n'.join(wrap(l, wrap_length)) for l in labels_list]
    return labels_list

In [73]:
def zip_five(df):
    '''
    Function
    --------
    Updates df with a "ZIP_FIVE" column (5 digit zip code)
    
    Parameters
    ----------
    df : Pandas DataFrame with a "ZIP" column
    
    Returns
    -------
    Pandas DataFrame with a ZIP_FIVE column (5 digit zip code)
    '''
    df['ZIP_FIVE'] = df['ZIP'].apply(lambda x: x[:5])
    return df

In [74]:
def find_same_zip(df, zip_code):
    '''
    Function
    --------
    Looks through Database and returns Entries with same zip code
    
    Parameters
    ----------
    df : Pandas DataFrame of Charities
    zip_code : Zip Code to Match on
    
    Returns
    -------
    Pandas DataFrame with Entries matching provided zip code
    '''
    zip_code = str(zip_code)
    
    temp_df = df[all_regions_df['ZIP_FIVE'] == zip_code]
    temp_df_two = df[all_regions_df['ZIP'] == zip_code]
    temp_df = pd.concat((temp_df,temp_df_two))
    
    return temp_df

In [75]:
def create_NTEE_dict(file):
    '''
    Function
    --------
    Create NTEE Dictionary from csv file
    
    Parameters
    ----------
    file : CSV file where first column is NTEE code
        and second column is NTEE code description
    
    Returns
    -------
    dictionary mapping NTEE code to NTEE code description
    '''
    NTEE_dict = dict()
    ntee_df = pd.read_csv(file,header=None)
    ntee_df = ntee_df.loc[:,:1]
    
    for index, row in ntee_df.iterrows():
        if type(row.loc[0]) != float:
            NTEE_dict[row.loc[0]] = row.loc[1].rstrip()
    
    return NTEE_dict

In [76]:
def create_major_categories_list(ntee_dict):
    '''
    Function
    --------
    Creates a list containing the major categories of NTEE codes
    
    Parameters
    ----------
    ntee_dict : Dictionary of NTEE codes and categories
    
    Returns
    -------
    list of major categories of NTEE codes
    '''
    major_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

    major_categories = []

    for let in major_letters:
        major_categories.append(ntee_dict[let])
    
    return major_categories

In [77]:
def create_NTEE_Categories(df,ntee_dict):
    '''
    Function
    --------
    Create NTEE_Major_Category and NTEE_Minor_Category and column for a Pandas DataFrame
    
    Parameters
    ----------
    df : Pandas Dataframe containing "NTEE_CD" Column
    ntee_dict : Dictionary containing NTEE Codes as Keys and
        NTEE Categories as Values
    
    Returns
    -------
    Pandas Dataframe with "NTEE_Major_Category" and "NTEE_Minor_Category" column
    '''
    major_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    
    df['NTEE_Major_Category'] = df['NTEE_CD'].apply(lambda x: ntee_dict[x[0]] if \
                                                    (type(x) == str and x[0] in major_letters) else x)
    
    df['NTEE_Minor_Category'] = df['NTEE_CD'].apply(lambda x: ntee_dict[x[:3]] if \
                                                    (type(x) == str and x[:3] in ntee_dict.keys()) else x)
    
    return df

In [78]:
def find_major_category(df,category:str):
    '''
    Function
    --------
    Filter a Pandas DataFrame by an NTEE Major Category
    
    Parameters
    ----------
    df : Pandas DataFrame with 'NTEE_Major_Category' column
    category : NTEE Category to Filter
    
    Return
    ------
    Filtered Pandas DataFrame
    '''
    temp_df = df[df['NTEE_Major_Category']==category]
    
    return temp_df

In [80]:
def search_by_zip_and_major_category(df, zipcode, category):
    '''
    Function
    --------
    Filter a Pandas DataFrame by Zip Code and NTEE Major Category
    
    Parameters
    ----------
    df : Pandas DataFrame with 'NTEE_Major_Category' column
    zipcode : Zipcode to Filter on
    category : NTEE Major Category to Filter on
    
    Return
    ------
    Filtered Pandas DataFrame
    '''
    
    temp_df = find_same_zip(df,zipcode)
    temp_df = find_major_category(temp_df,category)
    
    return temp_df

In [81]:
def find_minor_category(df,category:str):
    '''
    Function
    --------
    Filter a Pandas DataFrame by an NTEE Major Category
    
    Parameters
    ----------
    df : Pandas DataFrame with 'NTEE_Major_Category' column
    category : NTEE Category to Filter
    
    Return
    ------
    Filtered Pandas DataFrame
    '''
    temp_df = df[df['NTEE_Minor_Category']==category]
    
    return temp_df

In [82]:
def search_by_zip_and_minor_category(df, zipcode, category):
    '''
    Function
    --------
    Filter a Pandas DataFrame by zipcode and NTEE Minor Category
    
    Parameters
    ----------
    df : Pandas DataFrame with 'NTEE_Major_Category' column
    zipcode : Zipcode to Filter on
    category : NTEE Minor Category to Filter
    
    Return
    ------
    Filtered Pandas DataFrame    
    '''
    temp_df = find_same_zip(df,zipcode)
    temp_df = find_minor_category(temp_df,category)
    
    return temp_df

In [126]:
def add_county(df):
    '''
    Function
    --------
    Add a "county" column to Pandas DataFrame
    
    Parameters
    ----------
    df : Pandas DataFrame with 'ZIP_FIVE' column
    
    Return
    ------
    Pandas DataFrame with "County" column  
    '''
    print("Initializing SearchEngine")
    search = SearchEngine(simple_zipcode=True)
    print("Applying County Label")
    df['County'] = df['ZIP_FIVE'].apply(lambda x: search.by_zipcode(x).county)
    
    return df

In [15]:
region1_df = pd.read_csv('../data/eo1.csv')
region2_df = pd.read_csv('../data/eo2.csv')
region3_df = pd.read_csv('../data/eo3.csv')
region4_df = pd.read_csv('../data/eo3.csv')
all_regions_df = pd.concat([region1_df,region2_df,region3_df,region4_df])

In [16]:
all_regions_df = all_regions_df[all_regions_df["DEDUCTIBILITY"] == 1]

In [ ]:
# all_regions_df = all_regions_df.drop(\
#     columns=['FILING_REQ_CD',"PF_FILING_REQ_CD","STATUS","NTEE_CD"])

In [17]:
all_regions_df = zip_five(all_regions_df)

In [18]:
ntee_dict = create_NTEE_dict('../data/NTEE_file - Sheet1.csv')

In [19]:
ntee_list = create_major_categories_list(ntee_dict)

In [20]:
all_regions_df = create_NTEE_Categories(all_regions_df,ntee_dict)

In [42]:
all_regions_df = all_regions_df.reset_index(drop=True)

In [43]:
ntee_exists_list = list(all_regions_df['NTEE_CD'].dropna().index)

In [47]:
all_regions_df_NTEE = all_regions_df.loc[ntee_exists_list]

In [99]:
all_regions_df_NTEE.drop(columns = ['ICO','DEDUCTIBILITY','ACTIVITY','SORT_NAME'], inplace=True)

In [100]:
all_regions_df_NTEE.head()

,EIN,NAME,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,FOUNDATION,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category
2,587764,IGLESIA BETHESDA INC,157 ANDOVER ST,LOWELL,MA,01852-2348,0,3,3,7000,200401,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,01852,"Religion-Related, Spiritual Development",Protestant
3,635913,MINISTERIO APOSTOLICO JESUCRISTO ES EL SENOR INC,454 ESSEX ST,LAWRENCE,MA,01840-1242,0,3,3,7000,200401,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,01840,"Religion-Related, Spiritual Development",Protestant
4,765634,MERCY CHAPEL INTERNATIONAL,75 MORTON VILLAGE DR APT 408,MATTAPAN,MA,02126-2433,0,3,3,7000,200404,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,02126,"Religion-Related, Spiritual Development",Christian
5,841363,AGAPE HOUSE OF PRAYER,39 GOODALE RD STE 2,MATTAPAN,MA,02126-1527,0,3,3,1700,200412,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,02126,"Religion-Related, Spiritual Development",Christian
6,852649,BETHANY PRESBYTERIAN CHURCH,32 HARVARD ST,BROOKLINE,MA,02445-7994,0,3,3,7000,200504,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,02445,"Religion-Related, Spiritual Development",Christian


In [101]:
len(all_regions_df_NTEE)

1484617

In [54]:
manhasset_df = find_same_zip(all_regions_df_NTEE,11030)

/Users/jackvessa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/Users/jackvessa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [57]:
manhasset_df.head()

,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category
31765,61477860,PILUSO FOUNDATION,NaN,495 MANHASSET WOODS RD,MANHASSET,NY,11030-1663,0,3,3,1000,199703,1,4,602000000,2,1,201712.0,2,1,0,1,12,22990.0,2310.0,NaN,T70I,NaN,11030,"Philanthropy, Voluntarism and Grantmaking Foun...",Fund Raising Organizations That Cross Categories
33461,61633880,NEW LIGHT MISSION CHURCH,% THOMAS S YOO,1180 NORTHERN BLVD 2ND FLR,MANHASSET,NY,11030-3018,0,3,3,7000,200203,1,10,0,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,NaN,11030,"Religion-Related, Spiritual Development",Christian
36429,112116931,MANHASSET GREAT NECK ECONOMIC OPPORTUNITY COUNCIL,% ROBIN WHEAT,65 HIGH ST,MANHASSET,NY,11030-2906,0,3,3,1200,200008,1,15,0,1,1,201807.0,6,6,1,0,7,1304834.0,1013046.0,1003961.0,P99,NaN,11030,Human Services - Multipurpose and Other,Human Services - Multipurpose and Other N.E.C.
37590,112518200,AMERICAN ASSOCIATION FOR LABORATORY ANIMAL SCI...,NaN,350 COMMUNITY DR,MANHASSET,NY,11030-3816,0,3,3,1000,198109,1,16,995205000,1,1,201806.0,4,3,1,0,6,155406.0,58280.0,12439.0,D034,METROPOLITAN NY BRANCH,11030,Animal-Related,"Professional Societies, Associations"
37853,112560028,MANHASSET YOUTH COUNCIL INC,% PINZWOE PINZINN ESQS,PO BOX 238,MANHASSET,NY,11030-0238,0,3,3,1000,198108,1,15,328000000,1,1,201810.0,2,0,2,0,10,15193.0,0.0,0.0,O50Z,NaN,11030,Youth Development,"Youth Development Programs, Other"


In [58]:
find_major_category(manhasset_df,'Arts, Culture and Humanities')

,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category
38751,112688348,ONDERDONK LANDMARK SOCIETY LTD,% ONDERDONK LANDMARK SOCIETY,1471 NORTHERN BLVD,MANHASSET,NY,11030-3049,0,3,3,1000,201209,1,15,62000000,1,1,201812.0,0,0,2,0,12,0.0,0.0,0.0,A80,NaN,11030,"Arts, Culture and Humanities","Historical Societies, Related Historical Activ..."
43901,113377867,CHINESE AMERICAN ASSOCIATION OF NORTH HEMPSTEA...,NaN,PO BOX 523,MANHASSET,NY,11030-0523,0,3,3,2100,199810,1,16,119040093,1,1,201812.0,3,3,1,0,12,83532.0,62939.0,42056.0,A230,NaN,11030,"Arts, Culture and Humanities","Cultural, Ethnic Awareness"
44329,113417966,METROPOLITAN YOUTH ORCHESTRA SOCIETY OF NEW YO...,% ANTHEA JACKSON,13 MYRTLE ST,MANHASSET,NY,11030-2236,0,3,3,2000,199906,1,15,0,1,1,201806.0,4,5,1,0,6,179027.0,878127.0,878127.0,A69,NaN,11030,"Arts, Culture and Humanities",Symphony Orchestras
45655,113532550,ART GUILD OF PORT WASHINGTON INC,NaN,200 PT WA BLVD,MANHASSET,NY,11030-1713,0,3,3,2000,200111,1,16,0,1,1,201712.0,3,4,1,0,12,86119.0,150062.0,150062.0,A25,NaN,11030,"Arts, Culture and Humanities",Arts Education
47539,116037632,MANHASSET ART ASSOCIATION INC,% ANGELA TORO TREASURER,PO BOX 36,MANHASSET,NY,11030-0036,0,3,3,1000,201705,1,4,119059000,1,1,NaN,0,0,0,1,12,NaN,NaN,NaN,A26,NaN,11030,"Arts, Culture and Humanities",Arts Council/Agency
47595,116038976,SCIENCE MUSEUM OF LONG ISLAND,% ROSEDALE & SCERBO CPAS,1526 PLANDOME RD,MANHASSET,NY,11030-1038,0,3,3,2000,196411,1,16,59000000,1,1,201808.0,0,0,2,0,8,0.0,0.0,0.0,A520,NaN,11030,"Arts, Culture and Humanities",Children's Museums
57316,133936809,MELLIJOR CHARITABLE FOUNDATION INC,% WILLIAM BORDEN,220 ESTATES TER S,MANHASSET,NY,11030-4027,0,3,3,1000,199806,1,3,60000000,1,1,201712.0,1,1,0,1,12,1.0,1.0,NaN,A50Z,NaN,11030,"Arts, Culture and Humanities","Museum, Museum Activities"
137155,320476954,BELARUSIAN KRECEUSKI FOUNDATION,% VALERY DVORNIK,172 MILL SPRING RD,MANHASSET,NY,11030-3618,0,3,3,2000,201602,1,4,0,1,1,201712.0,4,2,0,1,12,134937.0,17851.0,NaN,A50,NaN,11030,"Arts, Culture and Humanities","Museum, Museum Activities"
146312,453843315,CREATIVE ARTS AS A GLOBAL RESOURSE INC,% WAYNE MIRSKY,1010 NORTHERN BLVD NO 208,MANHASSET,NY,11030-0000,0,3,3,1200,201306,1,15,0,1,1,201712.0,1,2,2,0,12,1738.0,16281.0,16281.0,A99,NaN,11030,"Arts, Culture and Humanities","Arts, Culture, and Humanities N.E.C."
153814,462941840,APPLE SEED FOUNDATION INC,NaN,47 PLANDOME RD,MANHASSET,NY,11030-2330,0,3,3,1000,201802,1,4,0,1,1,201712.0,1,2,0,1,12,9692.0,22000.0,NaN,A80,NaN,11030,"Arts, Culture and Humanities","Historical Societies, Related Historical Activ..."


## Rate By:
1. Narrow Down Categories
2. Charity Navigator Rating
3. Asset/Income
4. Region/State from ZipCode

In [59]:
from uszipcode import SearchEngine

In [60]:
search = SearchEngine(simple_zipcode=True)

In [124]:
search.by_zipcode('01852')

SimpleZipcode(zipcode='01852', zipcode_type='Standard', major_city='Lowell', post_office_city='Lowell, MA', common_city_list=['Lowell'], county='Middlesex County', state='MA', lat=42.63, lng=-71.3, timezone='Eastern', radius_in_miles=2.0, area_code_list=['781', '978'], population=33508, population_density=6765.0, land_area_in_sqmi=4.95, water_area_in_sqmi=0.24, housing_units=14430, occupied_housing_units=13481, median_home_value=236000, median_household_income=48951, bounds_west=-71.320025, bounds_east=-71.271271, bounds_north=42.656216, bounds_south=42.612675)

In [122]:
all_regions_df_NTEE

,EIN,NAME,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,FOUNDATION,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category
2,587764,IGLESIA BETHESDA INC,157 ANDOVER ST,LOWELL,MA,01852-2348,0,3,3,7000,200401,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,01852,"Religion-Related, Spiritual Development",Protestant
3,635913,MINISTERIO APOSTOLICO JESUCRISTO ES EL SENOR INC,454 ESSEX ST,LAWRENCE,MA,01840-1242,0,3,3,7000,200401,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,01840,"Religion-Related, Spiritual Development",Protestant
4,765634,MERCY CHAPEL INTERNATIONAL,75 MORTON VILLAGE DR APT 408,MATTAPAN,MA,02126-2433,0,3,3,7000,200404,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,02126,"Religion-Related, Spiritual Development",Christian
5,841363,AGAPE HOUSE OF PRAYER,39 GOODALE RD STE 2,MATTAPAN,MA,02126-1527,0,3,3,1700,200412,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,02126,"Religion-Related, Spiritual Development",Christian
6,852649,BETHANY PRESBYTERIAN CHURCH,32 HARVARD ST,BROOKLINE,MA,02445-7994,0,3,3,7000,200504,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,02445,"Religion-Related, Spiritual Development",Christian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2110311,996086871,WATERHOUSE CHARITABLE TR,770 KAPIOLANI BLVD STE 506,HONOLULU,HI,96813-5258,0,3,3,1000,200011,17,2,1,201806.0,8,6,1,0,6,35704565.0,2474123.0,1295808.0,T99,96813,"Philanthropy, Voluntarism and Grantmaking Foun...","Philanthropy, Voluntarism, and Grantmaking Fou..."
2110312,996087839,KEIKI KOHOLA PROJECT,1330 SABAL LAKES RD,DELRAY BEACH,FL,33445-1290,0,3,3,1000,200107,4,1,1,201712.0,2,2,0,3,12,10109.0,22000.0,NaN,U50,33445,"Science and Technology Research Institutes, Se...","Biological, Life Science Research"
2110313,996088748,UNOYO KOJIMA TRUST,PO BOX 3170,HONOLULU,HI,96802-3170,0,3,3,1000,200109,4,1,1,201803.0,4,3,0,1,3,377564.0,64829.0,NaN,T22,96802,"Philanthropy, Voluntarism and Grantmaking Foun...",Private Independent Foundations
2110314,996089401,TOYO SAKUMOTO CHARITABLE TR,PO BOX 3170,HONOLULU,HI,96802-3170,0,3,3,1000,200110,4,5,1,201903.0,5,3,0,1,3,660843.0,84812.0,NaN,T90,96802,"Philanthropy, Voluntarism and Grantmaking Foun...",Named Trusts/Foundations N.E.C.


In [125]:
%%time
add_county(all_regions_df_NTEE)

Initializing SearchEngine
Applying County Label


Exception during reset or similar
Traceback (most recent call last):
  File "/Users/jackvessa/opt/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 739, in _rollback_impl
    self.engine.dialect.do_rollback(self.connection)
  File "/Users/jackvessa/opt/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/default.py", line 530, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: Cannot operate on a closed database.

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/jackvessa/opt/anaconda3/lib/python3.7/site-packages/sqlalchemy/pool/base.py", line 680, in _finalize_fairy
    fairy._reset(pool)
  File "/Users/jackvessa/opt/anaconda3/lib/python3.7/site-packages/sqlalchemy/pool/base.py", line 865, in _reset
    self._reset_agent.rollback()
  File "/Users/jackvessa/opt/anaconda3/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1713, in rollback
    self._do_rollback(

CPU times: user 39min 1s, sys: 1min 16s, total: 40min 17s
Wall time: 42min 12s


,EIN,NAME,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,FOUNDATION,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category,County
2,587764,IGLESIA BETHESDA INC,157 ANDOVER ST,LOWELL,MA,01852-2348,0,3,3,7000,200401,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,01852,"Religion-Related, Spiritual Development",Protestant,Middlesex County
3,635913,MINISTERIO APOSTOLICO JESUCRISTO ES EL SENOR INC,454 ESSEX ST,LAWRENCE,MA,01840-1242,0,3,3,7000,200401,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,01840,"Religion-Related, Spiritual Development",Protestant,Essex County
4,765634,MERCY CHAPEL INTERNATIONAL,75 MORTON VILLAGE DR APT 408,MATTAPAN,MA,02126-2433,0,3,3,7000,200404,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,02126,"Religion-Related, Spiritual Development",Christian,Suffolk County
5,841363,AGAPE HOUSE OF PRAYER,39 GOODALE RD STE 2,MATTAPAN,MA,02126-1527,0,3,3,1700,200412,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,02126,"Religion-Related, Spiritual Development",Christian,Suffolk County
6,852649,BETHANY PRESBYTERIAN CHURCH,32 HARVARD ST,BROOKLINE,MA,02445-7994,0,3,3,7000,200504,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,02445,"Religion-Related, Spiritual Development",Christian,Norfolk County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2110311,996086871,WATERHOUSE CHARITABLE TR,770 KAPIOLANI BLVD STE 506,HONOLULU,HI,96813-5258,0,3,3,1000,200011,17,2,1,201806.0,8,6,1,0,6,35704565.0,2474123.0,1295808.0,T99,96813,"Philanthropy, Voluntarism and Grantmaking Foun...","Philanthropy, Voluntarism, and Grantmaking Fou...",Honolulu County
2110312,996087839,KEIKI KOHOLA PROJECT,1330 SABAL LAKES RD,DELRAY BEACH,FL,33445-1290,0,3,3,1000,200107,4,1,1,201712.0,2,2,0,3,12,10109.0,22000.0,NaN,U50,33445,"Science and Technology Research Institutes, Se...","Biological, Life Science Research",Palm Beach County
2110313,996088748,UNOYO KOJIMA TRUST,PO BOX 3170,HONOLULU,HI,96802-3170,0,3,3,1000,200109,4,1,1,201803.0,4,3,0,1,3,377564.0,64829.0,NaN,T22,96802,"Philanthropy, Voluntarism and Grantmaking Foun...",Private Independent Foundations,Honolulu County
2110314,996089401,TOYO SAKUMOTO CHARITABLE TR,PO BOX 3170,HONOLULU,HI,96802-3170,0,3,3,1000,200110,4,5,1,201903.0,5,3,0,1,3,660843.0,84812.0,NaN,T90,96802,"Philanthropy, Voluntarism and Grantmaking Foun...",Named Trusts/Foundations N.E.C.,Honolulu County


In [128]:
all_regions_df_NTEE.to_csv('All_Regions_w_Category_County_df')

In [129]:
x = pd.read_csv('All_Regions_w_Category_County_df')

In [130]:
x.head()

,Unnamed: 0,EIN,NAME,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,FOUNDATION,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category,County
0,2,587764,IGLESIA BETHESDA INC,157 ANDOVER ST,LOWELL,MA,01852-2348,0,3,3,7000,200401,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,1852,"Religion-Related, Spiritual Development",Protestant,Middlesex County
1,3,635913,MINISTERIO APOSTOLICO JESUCRISTO ES EL SENOR INC,454 ESSEX ST,LAWRENCE,MA,01840-1242,0,3,3,7000,200401,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,1840,"Religion-Related, Spiritual Development",Protestant,Essex County
2,4,765634,MERCY CHAPEL INTERNATIONAL,75 MORTON VILLAGE DR APT 408,MATTAPAN,MA,02126-2433,0,3,3,7000,200404,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,2126,"Religion-Related, Spiritual Development",Christian,Suffolk County
3,5,841363,AGAPE HOUSE OF PRAYER,39 GOODALE RD STE 2,MATTAPAN,MA,02126-1527,0,3,3,1700,200412,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,2126,"Religion-Related, Spiritual Development",Christian,Suffolk County
4,6,852649,BETHANY PRESBYTERIAN CHURCH,32 HARVARD ST,BROOKLINE,MA,02445-7994,0,3,3,7000,200504,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,2445,"Religion-Related, Spiritual Development",Christian,Norfolk County


In [ ]:
all_regions_df.groupby('NTEE_Major_Category').size().sort_values(ascending=False)

In [ ]:
search_by_zip_and_major_category(all_regions_df,11030,'Animal-Related')

In [ ]:
x = all_regions_df.loc[1]

In [ ]:
x

In [ ]:
x['city'] = x['ZIP_FIVE'].apply